In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext()
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/09 12:21:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/09 12:21:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/09 12:21:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/09 12:21:33 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0, "Tengo un poema escrito más de mil veces"),
    (1, "Ya no tengo la risa como un jilguero")],
 ["document", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
vectorizer  = CountVectorizer(inputCol="words", outputCol="rawFeatures")

idf = IDF(inputCol="rawFeatures", outputCol="features")

pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])

model = pipeline.fit(sentenceData)

In [18]:
import numpy as np

total_counts = model.transform(sentenceData)\
                    .select('rawFeatures').rdd\
                    .map(lambda row: row['rawFeatures'].toArray())\
                    .reduce(lambda x,y: [x[i]+y[i] for i in range(len(y))])

vocabList = model.stages[1].vocabulary
d = {'vocabList':vocabList,'counts':total_counts}

spark.createDataFrame(np.array(list(d.values())).T.tolist(),list(d.keys())).show()

+---------+------+
|vocabList|counts|
+---------+------+
|    tengo|   2.0|
|       un|   2.0|
|      mil|   1.0|
|       ya|   1.0|
|       la|   1.0|
|       de|   1.0|
|     risa|   1.0|
| jilguero|   1.0|
|       no|   1.0|
|    veces|   1.0|
|    poema|   1.0|
|     como|   1.0|
|      más|   1.0|
|  escrito|   1.0|
+---------+------+



In [19]:
from pyspark.sql.types import ArrayType, StringType

def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return udf(termsIdx2Term, ArrayType(StringType()))

vectorizerModel = model.stages[1]
vocabList = vectorizerModel.vocabulary
vocabList

['tengo',
 'un',
 'mil',
 'ya',
 'la',
 'de',
 'risa',
 'jilguero',
 'no',
 'veces',
 'poema',
 'como',
 'más',
 'escrito']

In [ ]:
#hecho a medias :/ 